In [1]:
#Use getpass to obtain user netID
import getpass
from os import lseek
import sys

# Import Libraries
from pyspark.sql import SparkSession, SQLContext
from pyspark.context import SparkContext
from pyspark.mllib.evaluation import RankingMetrics
from pyspark import SparkContext,SparkConf

import pandas as pd
import numpy as np
import random
import warnings
import pyspark
import sys

# Import Libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType, BooleanType
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.evaluation import RankingMetrics
from pyspark import SparkConf, SparkContext
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from math import sqrt

In [2]:
sc = SparkContext("local", "First App")

spark = SparkSession.builder.appName('popularity').getOrCreate()
ratings_train = spark.read.option("header",True).parquet('val_small_set.parquet')

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/17 12:51:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/17 12:51:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = ratings_train.toPandas()
# df['userId'] = df['userId'].sort_values()
df = df.sort_values(by = 'userId')
print(df)
user_movie = list(df.groupby('userId')['movieId'].apply(list))

       userId  timestamp  movieId  rating
10992       1  964983737     1206     5.0
10886       1  964982310     2273     4.0
10885       1  964982310     1408     3.0
10884       1  964982310      316     3.0
10883       1  964982290     2916     4.0
...       ...        ...      ...     ...
7610      597  941641222     2795     5.0
7611      597  941641264     2081     4.0
7612      597  941641264     2413     4.0
7603      597  941641183     1394     5.0
7536      597  941640075     2734     3.0

[15074 rows x 4 columns]


In [4]:
als_rec = spark.read.option("header",True).parquet('val_ALS_small_predicted_25_0.1_20.parquet')
rec = als_rec.toPandas()
rec = rec.sort_values(by = 'pr_userId')

In [5]:
print(rec)
rec = list(rec['rec_movie_id_indices'])
# rec = rec.groupby('pr_userId')['rec_movie_id_indices'].apply(list)
print('REC')
print(rec)

     pr_userId                               rec_movie_id_indices
12           1  [141718, 51931, 55363, 858, 3435, 26471, 2959,...
71           6  [2081, 2436, 457, 1939, 2563, 2398, 3675, 720,...
90           7  [1223, 55363, 1283, 3567, 3421, 3703, 1939, 62...
7           10  [71579, 92259, 2571, 115713, 55363, 141718, 96...
81          27  [3675, 2565, 62, 90866, 41566, 5014, 1081, 954...
..         ...                                                ...
74         586  [141718, 1376, 8132, 91529, 34405, 86377, 5536...
93         588  [89904, 3503, 296, 27611, 141718, 5915, 599, 3...
112        589  [8477, 3019, 71899, 3030, 866, 6818, 3341, 541...
47         594  [277, 2324, 68073, 720, 357, 918, 5014, 902, 4...
80         597  [141718, 3468, 8477, 858, 1203, 3030, 55363, 3...

[113 rows x 2 columns]
REC
[[141718, 51931, 55363, 858, 3435, 26471, 2959, 8477, 96004, 170355, 3379, 54503, 1213, 56782, 4235, 58998, 1394, 951, 26133, 65642, 8874, 6818, 6787, 475, 48516, 30745, 80906, 228

In [6]:
inp = list(zip(user_movie, rec))

In [7]:
rdd = sc.parallelize(inp)

metrics = RankingMetrics(rdd)

print("Precision 100:", metrics.precisionAt(100))
print("MAP 100:",metrics.meanAveragePrecisionAt(100))
print("NDCG 100:", metrics.ndcgAt(100))

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Precision 100: 0.036902654867256635
MAP 100: 0.006813195962653008
NDCG 100: 0.04451529627981667
